Run the two cells below.

In [ ]:
# @title Setting Up
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

import pandas as pd

In [ ]:
# @title Volunteer Dog Walking Solution
def optimize_dog_walking(time_required, available_time):
# Create the model
    model = ConcreteModel()

# Sets
    dogs = range(1, len(time_required) + 1)
    volunteers = range(1, len(available_time) + 1)

# Binary decision variables
    model.x = Var(dogs, volunteers, within=Binary)

# Objective function
    model.obj = Objective(expr=sum(time_required[d] * model.x[d, v] for d in dogs for v in volunteers), sense=minimize)

# Constraints
 # Each dog is walked by one volunteer
    model.walk_once = ConstraintList()
    for d in dogs:
        model.walk_once.add(sum(model.x[d, v] for v in volunteers) == 1)

 # Each required exercise time for a dog is less than the volunteer's available time
    model.volunteer_time = ConstraintList()
    for v in volunteers:
        model.volunteer_time.add(sum(time_required[d] * model.x[d, v] for d in dogs) <= available_time[v])

 # Each volunteer must walk at least one dog
    model.walk_at_least_one_dog = ConstraintList()
    for v in volunteers:
        model.walk_at_least_one_dog.add(sum(model.x[d, v] for d in dogs) >= 1)

 # Each volunteer can only walk at most three dogs
    model.walk_at_most_three_dogs = ConstraintList()
    for v in volunteers:
        model.walk_at_most_three_dogs.add(sum(model.x[d, v] for d in dogs) <= 3)

# Solve the model
    solver = SolverFactory('cbc')
    results = solver.solve(model)

# Display the results
    print("Objective Value: ", model.obj())
    print("\nVolunteer Assignments:")
    for d in dogs:
        for v in volunteers:
            if value(model.x[d, v]) == 1:
                print(f"Dog {d} is assigned to Volunteer {v}")


Run this third cell of code. Now, input the number of dogs you currently have at your shelter. Then, input the number of volunteers you currently have available. Then, enter the number of minutes each dog in the shelter requires for exercise. Last, enter how many minutes your volunteer has available to walk dogs. The solution will then output!

In [6]:
15# Get user inputs
num_dogs = int(input("Enter the number of dogs: "))
num_volunteers = int(input("Enter the number of volunteers: "))

time_required = {}
for d in range(1, num_dogs + 1):
    time_required[d] = int(input(f"Enter time required for Dog {d} (in minutes): "))

available_time = {}
for v in range(1, num_volunteers + 1):
    available_time[v] = int(input(f"Enter available time for Volunteer {v} (in minutes): "))

# Call the optimization function
optimize_dog_walking(time_required, available_time)



Enter the number of dogs: 10
Enter the number of volunteers: 5
Enter time required for Dog 1 (in minutes): 30
Enter time required for Dog 2 (in minutes): 28
Enter time required for Dog 3 (in minutes): 22
Enter time required for Dog 4 (in minutes): 45
Enter time required for Dog 5 (in minutes): 27
Enter time required for Dog 6 (in minutes): 18
Enter time required for Dog 7 (in minutes): 24
Enter time required for Dog 8 (in minutes): 27
Enter time required for Dog 9 (in minutes): 31
Enter time required for Dog 10 (in minutes): 41
Enter available time for Volunteer 1 (in minutes): 120
Enter available time for Volunteer 2 (in minutes): 120
Enter available time for Volunteer 3 (in minutes): 60
Enter available time for Volunteer 4 (in minutes): 45
Enter available time for Volunteer 5 (in minutes): 120
Objective Value:  293.0

Volunteer Assignments:
Dog 1 is assigned to Volunteer 1
Dog 2 is assigned to Volunteer 4
Dog 3 is assigned to Volunteer 5
Dog 4 is assigned to Volunteer 5
Dog 5 is assi